In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core import display as ICD
import seaborn as sns
import glob
import scipy
import os
pd.set_option('display.max_columns', 100)
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
DATA_FOLDER = './avg_data/'
os.listdir(DATA_FOLDER)
all_anem=glob.glob(DATA_FOLDER + "anem*")
mast_anem=all_anem[:6]
top_anem=all_anem[6]
only_mag=True
h=np.arange(1.5,22,4)
df_list=[]
for idx,i in enumerate(mast_anem):
    df_temp=pd.read_csv(i, sep=" ", header=None, index_col=0, names=['u','direction'])
    df_temp['h']=h[idx]
    df_list.append(df_temp)
mast_anem_df = pd.concat(df_list)
rad_df = pd.read_csv(DATA_FOLDER+'radiometer.csv', sep=" ",index_col=0)
temp_df = pd.read_csv(DATA_FOLDER+'surf_temp.txt', sep=" ",index_col=0)
top_anem_df = pd.read_csv(top_anem, sep=" ",index_col=0,names=['u_top','direction_top'])
time_var_df = top_anem_df.merge(rad_df,how='left', left_index=True,right_index=True)
time_var_df = time_var_df.merge(temp_df,how='left', left_index=True,right_index=True)
tot_df=mast_anem_df.merge(time_var_df,how='left', left_index=True,right_index=True)
tot_df = tot_df.dropna(axis=0, how='any')
tot_df = tot_df.drop(columns=['Albedo [-]'])
tot_df

,u,direction,h,u_top,direction_top,Pyranometer Upper Irradiance [W/m$^2$],Pyranometer Lower Irradiance [W/m$^2$],Pyrgeometer Upper Irradiance [W/m$^2$],Pyrgeometer Lower Irradiance [W/m$^2$],Net Solar radiation [W/m$^2$],Net (total) radiation [W/m$^2$],Net Far Infrared radiation [W/m$^2$],Sky temperature [°C],Radiometer Ground temperature [°C],Sensor Ground temperature [°C],North temperature [°C],East temperature [°C],South temperature [°C],West temperature [°C]
2018-11-14 00:00:00,0.66,160.0,1.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:00:00,0.91,225.0,5.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:00:00,0.57,185.0,9.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:00:00,2.64,284.0,13.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:00:00,2.98,313.0,17.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:00:00,2.71,328.0,21.5,2.76,298.0,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:01:00,0.69,187.0,1.5,2.61,316.0,2.2,-0.0,371.7,365.8,2.2,8.1,5.9,11.4,10.3,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:01:00,0.99,272.0,5.5,2.61,316.0,2.2,-0.0,371.7,365.8,2.2,8.1,5.9,11.4,10.3,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:01:00,0.61,248.0,9.5,2.61,316.0,2.2,-0.0,371.7,365.8,2.2,8.1,5.9,11.4,10.3,10.5,13.1,11.8,11.6,10.5
2018-11-14 00:01:00,2.25,281.0,13.5,2.61,316.0,2.2,-0.0,371.7,365.8,2.2,8.1,5.9,11.4,10.3,10.5,13.1,11.8,11.6,10.5


### Built in house categorization

In [ ]:
def categorization_index(feature, bin_):
    feature = np.array(feature)
    bin_ = np.array(bin_)
    feature_cat = np.copy(feature)
    for i in range(0,feature.shape[1]):
        cut = pd.cut(feature[:,i],bin_[i])
        index = cut.codes
        feature_cat[:,i] = index
    return feature_cat

In [ ]:
bin_ = 20*np.ones(19)
data = tot_df.iloc[:,0:-1]
cat = categorization_index(data, bin_)
plt.figure()
plt.plot(cat[:,0])
plt.figure()
plt.plot(np.array(data)[:,0])
cat = cat.astype(int)
print cat

In [ ]:
train = np.array(cat[0:-1])
train.shape

#### Wind speed direction decomposition

Transform absolute value and direction in vector components

In [3]:
# create columns with coordinate velocities output
tot_df['u_x']=tot_df['u']*np.cos(np.radians(tot_df['direction']))
tot_df['u_y']=tot_df['u']*np.sin(np.radians(tot_df['direction']))

In [4]:
# create columns with coordinate velocities input top mast anemometer
tot_df['u_top_x']=tot_df['u_top']*np.cos(np.radians(tot_df['direction_top']))
tot_df['u_top_y']=tot_df['u_top']*np.sin(np.radians(tot_df['direction_top']))

In [5]:
# drop the columns which are not used anymore
tot_df=tot_df.drop(columns=['u', 'u_top', 'direction', 'direction_top'])

In [6]:
tot_df.head()

,h,Pyranometer Upper Irradiance [W/m$^2$],Pyranometer Lower Irradiance [W/m$^2$],Pyrgeometer Upper Irradiance [W/m$^2$],Pyrgeometer Lower Irradiance [W/m$^2$],Net Solar radiation [W/m$^2$],Net (total) radiation [W/m$^2$],Net Far Infrared radiation [W/m$^2$],Sky temperature [°C],Radiometer Ground temperature [°C],Sensor Ground temperature [°C],North temperature [°C],East temperature [°C],South temperature [°C],West temperature [°C],u_x,u_y,u_top_x,u_top_y
2018-11-14 00:00:00,1.5,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5,-0.620197,0.225733,1.295742,-2.436935
2018-11-14 00:00:00,5.5,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5,-0.643467,-0.643467,1.295742,-2.436935
2018-11-14 00:00:00,9.5,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5,-0.567831,-0.049679,1.295742,-2.436935
2018-11-14 00:00:00,13.5,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5,0.638674,-2.561581,1.295742,-2.436935
2018-11-14 00:00:00,17.5,1.8,-2.9,369.0,364.7,4.7,9.0,4.3,10.9,10.0,10.5,13.1,11.8,11.6,10.5,2.032355,-2.179434,1.295742,-2.436935


### Splitting the data

In [8]:
X = np.array(tot_df.drop(columns=['u_x', 'u_y']))
y = np.array(tot_df[['u_x', 'u_y']])

In [9]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

### Categorization with scikit

In [12]:
est = KBinsDiscretizer(n_bins=[20, 20], encode='ordinal').fit(y_tr)
y_trans=est.transform(y_tr)  

## Neural Network Implementation 2

In [13]:
import torch
import torch.nn as nn

Considerations on Batch Size:

For instance, let's say you have 1050 training samples and you want to set up a batch_size equal to 100. The algorithm takes the first 100 samples (from 1st to 100th) from the training dataset and trains the network. Next it takes the second 100 samples (from 101st to 200th) and trains the network again. We can keep doing this procedure until we have propagated through all samples of the network. A problem usually happens with the last set of samples. In our example we've used 1050 which is not divisible by 100 without remainder. The simplest solution is just to get the final 50 samples and train the network.

Advantages of using a batch size < number of all samples:

It requires less memory. Since you train the network using fewer samples, the overall training procedure requires less memory. That's especially important if you are not able to fit the whole dataset in your machine's memory.
Typically networks train faster with mini-batches. That's because we update the weights after each propagation. In our example we've propagated 11 batches (10 of them had 100 samples and 1 had 50 samples) and after each of them we've updated our network's parameters. If we used all samples during propagation we would make only 1 update for the network's parameter.
Disadvantages of using a batch size < number of all samples:

The smaller the batch the less accurate the estimate of the gradient will be. In the figure below you can see that the direction of the mini-batch gradient (green color) fluctuates much more in comparison to the direction of the full batch gradient (blue color).

[link](https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network)

In [31]:
# define the size of the layer and the batch size

# n_in is the shape of the feature matrix X
# n_h is the number of layers 
# n_out is the number of output
# batch_size defines the number of samples that will be propagated through the network.

n_in, n_h, n_out, batch_size = X_tr.shape[1], 5, y_tr.shape[1], 50

Transform the numpy array in Pytorch arrays

In [48]:
x_train = torch.from_numpy(X_tr)
y_train = torch.from_numpy(y_trans) 

In [40]:
y_train

tensor([[ 7., 17.],
        [ 8., 15.],
        [18., 19.],
        ...,
        [14.,  7.],
        [ 5., 14.],
        [18.,  0.]], dtype=torch.float64)

This creates a model that looks like input -> linear -> relu -> linear -> sigmoid.

In [41]:
model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out),
                     nn.Sigmoid())

Construct Loss function

In [42]:
criterion = torch.nn.MSELoss()

Define our optimizer. We use Stochastic Gradient Descent 

In [43]:
# lr is the learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Train the network for 50 epochs. This does the forward propagation, loss computation, backward propagation and parameter updation in that sequence.

In [46]:
for epoch in range(50):
    # Forward Propagation
    y_pred = model(x_train.long())
    # Compute and print loss
    loss = criterion(y_pred, y_train.long())
    print('epoch: ', epoch,' loss: ', loss.item())
    # Zero the gradients
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

RuntimeError: Expected object of type torch.FloatTensor but found type torch.LongTensor for argument #4 'mat1'

## Neural Network Implementation 1

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F      # activation function
import matplotlib.pyplot as plt

In [ ]:
x_train = train[:,3]
print x_train.shape
y_train = np.array(train[:,1])
print y_train.shape
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
print y_train.shape

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)  
        self.predict = torch.nn.Linear(n_hidden, n_output)  

    def forward(self, x):
        a1 = F.relu(self.hidden(x)) 
        #a1 = torch.sigmoid(self.hidden(x))
        #a1 = self.hidden(x) # 隐藏层用 relu
        y = self.predict(a1)
        return y

net = Net(n_feature=1, n_hidden=10, n_output=1)
print(net)

In [ ]:
loss_func = torch.nn.MSELoss()   
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)

In [ ]:
x_train, y_train = Variable(x_train), Variable(y_train)
x_train = x_train.type(torch.FloatTensor)
y_train = y_train.type(torch.FloatTensor)
print x_train
x_train = Variable(x_train)
y_train = Variable(y_train)
print x_train.size()
print y_train.size()
y_train = y_train.view(8459, 1) 
x_train = x_train.view(8459, 1)
print y_train

In [ ]:
for t in range(1000):
    prediction = net(x_train)
    loss = loss_func(prediction, y_train)

    optimizer.zero_grad()    # clear gradients for next train
    loss.backward()          # backpropagation
    optimizer.step()         # update（w、b）

In [ ]:
print loss
net.eval()
predict = net(Variable(x_train))
predict = predict.data.numpy()

In [ ]:
print np.unique(predict)

In [ ]:
plt.plot(predict)

In [ ]:
plt.plot(np.array(y_train))